In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import re
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
rmas = pd.read_csv('bofarmareport1222024.csv', encoding = 'unicode_escape', parse_dates=['Created Date', 'Date Submitted', 'Date/Time Opened'])

# Combining data from different columns into one single narrative to facilitate searching
rmas["General Details"] = rmas["Description"] + rmas["Subject"] + rmas["Technical Failure Description"]

In [4]:
rmas.columns

Index(['Date/Time Opened', 'Created Date', 'Date/Time Closed', 'Account Name',
       'Description', 'Subject', 'Technical Failure Description',
       'Case Number', 'RMA Case Number', 'Customer Selected Serial Number',
       'Returning Serial Number', 'Replacement Serial Number', 'RMA Tracking',
       'RMA City', 'RMA State / Province', 'RMA Shipping Address', 'RMA ID',
       'RMA Number', 'RMA Status', 'RMA Zip / Postal Code',
       'RMA Shipping Country', 'RMA 15 Minute Time Trigger',
       'DOA RMA Replacement', 'RMA Destination City',
       'RMA Destination Company', 'RMA Destination Contact Email',
       'RMA Destination Contact Phone', 'RMA Destination Contact',
       'RMA Destination Country', 'RMA Destination Postal Code',
       'RMA Destination State', 'RMA Destination Street Address',
       'RMA Return Item Height', 'RMA Tracking Number', 'RMA Type',
       'Last Modified Date', 'Last Activity Date', 'Date Submitted',
       'RMA Shipping Street 1', 'Time Open to 

In [5]:
rmas.head(5)

,Date/Time Opened,Created Date,Date/Time Closed,Account Name,Description,Subject,Technical Failure Description,Case Number,RMA Case Number,Customer Selected Serial Number,...,Case Origin,Type,Replacement Unit Type,Support Type,OS Release,Product,Send Return Instructions,Returning Model,Device will not be returned,General Details
0,2023-12-31 09:47:00,2023-12-31,1/4/2024 8:24 PM,Bank of America,Title: ION RMA for bad internet 1 port\nPID: ...,Title: ION RMA for bad internet 1 port PID: I...,Reason for Return: Internet port is bad and ne...,2839415,2839415,30-005948-4398,...,Web,Tech Support,Refurbished,4HR,NaN,ION 3000,0,PAN-CG-ION-3000,0,Title: ION RMA for bad internet 1 port\nPID: ...
1,2023-12-31 09:23:00,2024-01-01,1/4/2024 8:12 AM,Bank of America,Title: ION RMA for bad internet 1 port\nPID: ...,Title: ION RMA for bad internet 1 port PID: I...,Title: ION RMA for bad internet 1 port PID: I...,2839411,2839411,30-010256-8422,...,Web,Tech Support,Refurbished,4HR,NaN,ION 3000,0,PAN-CG-ION-3000,0,Title: ION RMA for bad internet 1 port\nPID: ...
2,2023-12-31 09:06:00,2023-12-31,1/4/2024 7:33 AM,Bank of America,Title: ION RMA for bad internet 1 port\nPID: ...,Title: ION RMA for bad internet 1 port PID: I...,Subject\nTitle: ION RMA for bad internet 1 por...,2839408,2839408,30-005948-4398,...,Web,Tech Support,Refurbished,4HR,NaN,ION 3000,0,PAN-CG-ION-3000,0,Title: ION RMA for bad internet 1 port\nPID: ...
3,2023-12-25 07:46:00,2023-12-25,1/1/2024 4:40 PM,Bank of America,"ION 3000 reloading constantly, no issues found...","ION 3000 reloading constantly, no issues found...",Failure symptom technical summary: Having prob...,2835234,2835234,CANNOT FIND S/N,...,Web,Tech Support,Refurbished,Premium,NaN,NaN,0,PAN-CG-ION-3000,0,"ION 3000 reloading constantly, no issues found..."
4,2023-12-20 22:27:00,2023-12-21,12/26/2023 7:30 PM,Bank of America,Controller 1 port faulty Need RMA as per PA en...,Controller 1 port faulty Need RMA as per PA en...,Reason for Return: Controller 1 port faulty Ne...,2832127,2832127,30-007569-3586,...,Web,Tech Support,Refurbished,4HR,NaN,ION 3000,0,PAN-CG-ION-3000,0,Controller 1 port faulty Need RMA as per PA en...


In [6]:
rmas['Product']

0      ION 3000
1      ION 3000
2      ION 3000
3           NaN
4      ION 3000
         ...   
516    ION 3000
517    ION 3000
518    ION 3000
519    ION 3000
520    ION 3000
Name: Product, Length: 521, dtype: object

In [7]:
# Filter records for IONs only
# ion_rmas = rmas.loc[rmas["Product"].str.contains("ION"), :]

In [8]:
rmas.columns

Index(['Date/Time Opened', 'Created Date', 'Date/Time Closed', 'Account Name',
       'Description', 'Subject', 'Technical Failure Description',
       'Case Number', 'RMA Case Number', 'Customer Selected Serial Number',
       'Returning Serial Number', 'Replacement Serial Number', 'RMA Tracking',
       'RMA City', 'RMA State / Province', 'RMA Shipping Address', 'RMA ID',
       'RMA Number', 'RMA Status', 'RMA Zip / Postal Code',
       'RMA Shipping Country', 'RMA 15 Minute Time Trigger',
       'DOA RMA Replacement', 'RMA Destination City',
       'RMA Destination Company', 'RMA Destination Contact Email',
       'RMA Destination Contact Phone', 'RMA Destination Contact',
       'RMA Destination Country', 'RMA Destination Postal Code',
       'RMA Destination State', 'RMA Destination Street Address',
       'RMA Return Item Height', 'RMA Tracking Number', 'RMA Type',
       'Last Modified Date', 'Last Activity Date', 'Date Submitted',
       'RMA Shipping Street 1', 'Time Open to 

In [9]:
week_days = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
months_year = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

In [10]:
def find_serial_number(astring):
    '''
    Return the SN as a string from ASTRING
    '''
    pattern = r'\d{2}-\d{6}-\d{4}'
    try:
        sn = re.findall(pattern, astring)
        if len(sn) > 0:
            return sn[0]
    except TypeError:
        return None

In [11]:
def find_version(astring):
    '''
    Return the software version from ASTRING
    '''
    pattern = r'\d\.\d\.\d[-]?[b]?\d?'
    try:
        sn = re.findall(pattern, astring)
        if len(sn) > 0:
            return sn[0]
    except TypeError:
        return None    

In [12]:
def find_failure(astring):
    '''
    Return characters in ASTRING after Issue description
    '''
    try:
        sn0 = re.findall('Issue description:([\s]?.*)', astring)
        sn1 = re.findall('Description of Issue:([\s]?.*)', astring)
        sn2 = re.findall('Reason for Return:([\s]?.*)', astring)
        if len(sn0) > 0:
            return sn0[0].strip()
        if len(sn1) > 0:
            return sn1[0].strip()  
        else:
            if len(sn2) > 0:
                return sn2[0].strip()
    except TypeError:
        return None

In [13]:
def find_POD(astring):
    '''
    Return characters in ASTRING after Issue description
    '''
    try:
        pod = re.findall('POD Date Time :([\s]?.*)', astring)
        if len(pod) > 0:
            return pod[0].strip()
    except TypeError:
        return None

In [14]:
def POD_to_datetime(astring):
    '''
    Return a datetime format of ASTRING
    '''
    if astring != None:
        dt = datetime.strptime(astring, '%Y-%m-%d %H:%M:%S')
        return dt
    else:
        dt = datetime.strptime('1999-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
        return dt

In [15]:
def non_zero(aseries, bseries):
    '''
    Return only the values of BSERIES that are greater than zero
    '''
    afiltered = []
    bfiltered = []
    for x, y in zip(aseries, bseries):
        if y > 0:
            afiltered.append(x)
            bfiltered.append(y)
            
    return afiltered, bfiltered
        

In [16]:
rmas["Defective SN"] = rmas["General Details"].apply(lambda x: find_serial_number(x))
rmas["Replacement SN"] = rmas["RMA Tracking"].apply(lambda x: find_serial_number(x))
rmas["Defective Version"] = rmas["General Details"].apply(lambda x: find_version(x))
rmas["Issue"] = rmas["General Details"].apply(lambda x: find_failure(x))
rmas["Year"] = rmas["Date/Time Opened"].dt.year
rmas["Month"] = rmas["Date/Time Opened"].dt.month
rmas["Day"] = rmas["Date/Time Opened"].dt.day
rmas["Weekday"] = rmas["Date/Time Opened"].dt.weekday
rmas["Hour"] = rmas["Date/Time Opened"].dt.hour
rmas["Minute"] = rmas["Date/Time Opened"].dt.minute
rmas["POD"] = rmas["RMA Tracking"].apply(lambda x: find_POD(x))
rmas["Date_POD"] = rmas["POD"].apply(lambda x: POD_to_datetime(x))
rmas["Year_POD"] = rmas["Date_POD"].apply(lambda x: x.year)
rmas["Month_POD"] = rmas["Date_POD"].apply(lambda x: x.month)
rmas["Day_POD"] = rmas["Date_POD"].apply(lambda x: x.day)
rmas["Weekday_POD"] = rmas["Date_POD"].apply(lambda x: x.weekday())
rmas["Hour_POD"] = rmas["Date_POD"].apply(lambda x: x.hour)
rmas["Minute_POD"] = rmas["Date_POD"].apply(lambda x: x.minute)
rmas["Time_Delta"] = rmas["Date_POD"] - rmas["Date/Time Opened"]
rmas["Hours_Delta"] = rmas["Time_Delta"].apply(lambda x: x.seconds // 3600)

In [17]:
summary_rmas = rmas[["Date/Time Opened", "Case Number", "Product", "RMA Status", "Defective Version", "Defective SN", "Replacement SN", 
                    "Issue", "RMA Shipping Country", "RMA State / Province", "Year", "Month", "Day", "Weekday", "Hour", "Minute", 
                    "Year_POD", "Month_POD", "Day_POD", "Weekday_POD", "Hour_POD", "Minute_POD", "Hours_Delta"]]

In [18]:
summary_rmas

,Date/Time Opened,Case Number,Product,RMA Status,Defective Version,Defective SN,Replacement SN,Issue,RMA Shipping Country,RMA State / Province,...,Weekday,Hour,Minute,Year_POD,Month_POD,Day_POD,Weekday_POD,Hour_POD,Minute_POD,Hours_Delta
0,2023-12-31 09:47:00,2839415,ION 3000,Requested,None,30-009148-5679,30-016645-9979,Internet port is bad and need RMA of the whole...,United States,New York,...,6,9,47,2024,1,2,1,9,9,23
1,2023-12-31 09:23:00,2839411,ION 3000,Requested,None,30-010256-8422,None,None,United States,New Mexico,...,6,9,23,1999,1,1,4,0,0,14
2,2023-12-31 09:06:00,2839408,ION 3000,Requested,None,30-005948-4398,30-005841-9728,None,United States,New Jersey,...,6,9,6,2024,1,2,1,9,1,23
3,2023-12-25 07:46:00,2835234,NaN,Requested,None,30-017102-4897,30-012699-7368,The device is up faulty.,United States,Florida,...,0,7,46,2023,12,27,2,10,17,2
4,2023-12-20 22:27:00,2832127,ION 3000,Requested,None,None,None,Controller 1 port faulty Need RMA,United States,Florida,...,2,22,27,1999,1,1,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,2021-03-31 17:06:00,1785533,ION 3000,Requested,None,None,None,None,United States,Minnesota,...,2,17,6,1999,1,1,4,0,0,6
517,2021-03-29 14:08:00,1782848,ION 3000,Requested,None,30-010073-8788,None,"ION 3000, device doesn't boot anymore. Shows E...",United States,Minnesota,...,0,14,8,1999,1,1,4,0,0,9
518,2021-03-29 13:38:00,1782799,ION 3000,Requested,None,30-009405-3547,None,None,United States,MN,...,0,13,38,1999,1,1,4,0,0,10
519,2021-03-05 10:02:00,1758699,ION 3000,Requested,None,30-010018-6729,None,None,United States,Minnesota,...,4,10,2,1999,1,1,4,0,0,13


In [19]:
# only Requested RMAs
summary_rmas = summary_rmas.loc[rmas["RMA Status"] == "Requested", :]

In [20]:
len(summary_rmas)

458

In [21]:
# drop duplicate case numbers
summary_rmas = summary_rmas.drop_duplicates(subset="Case Number")
summary_rmas.replace({'Weekday': {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}}, inplace=True)
summary_rmas.replace({'Weekday_POD': {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}}, inplace=True)
summary_rmas.replace({'Month': {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}}, inplace=True)
len(summary_rmas)

448

In [22]:
summary_rmas.columns

Index(['Date/Time Opened', 'Case Number', 'Product', 'RMA Status',
       'Defective Version', 'Defective SN', 'Replacement SN', 'Issue',
       'RMA Shipping Country', 'RMA State / Province', 'Year', 'Month', 'Day',
       'Weekday', 'Hour', 'Minute', 'Year_POD', 'Month_POD', 'Day_POD',
       'Weekday_POD', 'Hour_POD', 'Minute_POD', 'Hours_Delta'],
      dtype='object')

In [23]:
summary_rmas

,Date/Time Opened,Case Number,Product,RMA Status,Defective Version,Defective SN,Replacement SN,Issue,RMA Shipping Country,RMA State / Province,...,Weekday,Hour,Minute,Year_POD,Month_POD,Day_POD,Weekday_POD,Hour_POD,Minute_POD,Hours_Delta
0,2023-12-31 09:47:00,2839415,ION 3000,Requested,None,30-009148-5679,30-016645-9979,Internet port is bad and need RMA of the whole...,United States,New York,...,Sunday,9,47,2024,1,2,Tuesday,9,9,23
1,2023-12-31 09:23:00,2839411,ION 3000,Requested,None,30-010256-8422,None,None,United States,New Mexico,...,Sunday,9,23,1999,1,1,Friday,0,0,14
2,2023-12-31 09:06:00,2839408,ION 3000,Requested,None,30-005948-4398,30-005841-9728,None,United States,New Jersey,...,Sunday,9,6,2024,1,2,Tuesday,9,1,23
3,2023-12-25 07:46:00,2835234,NaN,Requested,None,30-017102-4897,30-012699-7368,The device is up faulty.,United States,Florida,...,Monday,7,46,2023,12,27,Wednesday,10,17,2
4,2023-12-20 22:27:00,2832127,ION 3000,Requested,None,None,None,Controller 1 port faulty Need RMA,United States,Florida,...,Wednesday,22,27,1999,1,1,Friday,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,2021-03-31 17:06:00,1785533,ION 3000,Requested,None,None,None,None,United States,Minnesota,...,Wednesday,17,6,1999,1,1,Friday,0,0,6
517,2021-03-29 14:08:00,1782848,ION 3000,Requested,None,30-010073-8788,None,"ION 3000, device doesn't boot anymore. Shows E...",United States,Minnesota,...,Monday,14,8,1999,1,1,Friday,0,0,9
518,2021-03-29 13:38:00,1782799,ION 3000,Requested,None,30-009405-3547,None,None,United States,MN,...,Monday,13,38,1999,1,1,Friday,0,0,10
519,2021-03-05 10:02:00,1758699,ION 3000,Requested,None,30-010018-6729,None,None,United States,Minnesota,...,Friday,10,2,1999,1,1,Friday,0,0,13


In [24]:
summary_rmas.to_excel("bofa_summary_rmas.xlsx")

In [ ]:
summary_rmas.to_csv("summary_rmas.csv")

In [ ]:
summary_rmas = pd.read_csv("summary_rmas.csv")

In [ ]:
summary_rmas

In [ ]:
# visualization of grouped values
pd.set_option('display.max_rows', None)
agg_rmas_prod_month_2023 = summary_rmas.loc[summary_rmas["Year"] == 2023].groupby(by=["Product", "Month"], sort=True).agg(
                            Count = pd.NamedAgg("Product", aggfunc="count"))
agg_rmas_prod_month_2023 = agg_rmas_prod_month_2023.reset_index()
# the pivot method allows to convert the grouped data to a table
agg_pivot_2023 = agg_rmas_prod_month_2023.pivot(index="Month", columns="Product", values="Count").fillna(0)


agg_rmas_prod_month_2022 = summary_rmas.loc[summary_rmas["Year"] == 2022].groupby(by=["Product", "Month"], sort=True).agg(
                            Count = pd.NamedAgg("Product", aggfunc="count"))
agg_rmas_prod_month_2022 = agg_rmas_prod_month_2022.reset_index()
agg_pivot_2022 = agg_rmas_prod_month_2022.pivot(index="Month", columns="Product", values="Count").fillna(0)


agg_rmas_prod_month_2021 = summary_rmas.loc[summary_rmas["Year"] == 2021].groupby(by=["Product", "Month"], sort=True).agg(
                            Count = pd.NamedAgg("Product", aggfunc="count"))
agg_rmas_prod_month_2021 = agg_rmas_prod_month_2021.reset_index()
agg_pivot_2021 = agg_rmas_prod_month_2021.pivot(index="Month", columns="Product", values="Count").fillna(0)


In [ ]:
agg_pivot_2021

In [ ]:
agg_pivot_2022

In [ ]:
agg_pivot_2023

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(15,5))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=None)

axes[0].set_xlim([1, 12])
axes[0].set_ylim([0, 20])

for i in [0, 1, 2]:
    axes[i].set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
    axes[i].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)

y_ticks = [x for x in range(0, 15, 1)]
axes[0].set_yticks(y_ticks)
axes[0].set_ylabel("Number of IONs")

# 
ax, ay = non_zero(agg_pivot_2021.index, agg_pivot_2021.iloc[:,0])
# bx, by = non_zero(agg_pivot_2021.index, agg_pivot_2021.iloc[:,1])
# cx, cy = non_zero(agg_pivot_2021.index, agg_pivot_2021.iloc[:,2])

"""
# axes[0].scatter(x=ax, y=ay, color="purple", label="ION 1000")
axes[0].scatter(x=ax, y=ay, color="blue", label="ION 2000")
#axes[0].scatter(x=bx, y=by, color="orange", label="ION 7000")
# axes[0].scatter(x=cx, y=cy, color="green", label="ION 9000")
axes[0].grid(alpha=0.5, linestyle='dotted')
axes[0].legend()


#
ax, ay = non_zero(agg_pivot_2022.index, agg_pivot_2022.iloc[:,0])
#bx, by = non_zero(agg_pivot_2022.index, agg_pivot_2022.iloc[:,1])
#cx, cy = non_zero(agg_pivot_2022.index, agg_pivot_2022.iloc[:,2])

# axes[1].scatter(x=ax, y=ay, color="purple", label="ION 1000")
axes[1].scatter(x=ax, y=ay, color="blue", label="ION 2000")
#axes[1].scatter(x=bx, y=by, color="orange", label="ION 7000")
#axes[1].scatter(x=cx, y=cy, color="green", label="ION 9000")
#axes[1].grid(alpha=0.5, linestyle='dotted')
axes[1].legend()


#
ax, ay = non_zero(agg_pivot_2023.index, agg_pivot_2023.iloc[:,0])
bx, by = non_zero(agg_pivot_2023.index, agg_pivot_2023.iloc[:,1])
#cx, cy = non_zero(agg_pivot_2023.index, agg_pivot_2023.iloc[:,2])

axes[2].scatter(x=ax, y=ay, color="purple", label="ION 2000")
axes[2].scatter(x=bx, y=by, color="blue", label="ION 3000")
#axes[2].scatter(x=cx, y=cy, color="orange", label="ION 7000")
#axes[2].scatter(x=cx, y=cy, color="green", label="ION 9000")
axes[2].grid(alpha=0.5, linestyle='dotted')
axes[2].legend()


# Changing title font properties
title_font = {
    'family': 'serif',   # Font family
    'size': 14,          # Font size
    'weight': 'bold',     # Font weight
    'color': 'gray'
}
axes[0].set_title(label="# of RMAs in 2021 by Month", fontdict=title_font)
axes[1].set_title(label="# of RMAs in 2022 by Month", fontdict=title_font)
axes[2].set_title(label="# of RMAs in 2023 by Month", fontdict=title_font)
axes[0].set_xlabel("Months of the Year")
axes[1].set_xlabel("Months of the Year")
axes[2].set_xlabel("Months of the Year");
"""

In [ ]:
# visualization of grouped values
pd.set_option('display.max_rows', None)
agg_rmas_day_2023 = summary_rmas.loc[summary_rmas["Year"] == 2023].groupby(by=["Month", "Weekday_POD"], sort=True).agg(
                            Count = pd.NamedAgg("Weekday_POD", aggfunc="count"))

agg_rmas_day_2023 = agg_rmas_day_2023.reset_index()
# the pivot method allows to convert the grouped data to a table
# agg_day_pivot_2023 = agg_rmas_day_2023.pivot(index="Month", columns="Weekday_POD", values="Count").fillna(0)
agg_day_pivot_2023

In [ ]:
agg_day_pivot_2023.values

In [ ]:
sns.catplot(data=agg_day_pivot_2023.values[0], x=agg_day_pivot_2023.columns, y=agg_day_pivot_2023.index[0])

In [ ]:
sns.catplot(x=agg_day_pivot_2023.columns, y="tip_pct", hue="time", col="smoker",
   .....:             kind="bar", data=tips[tips.tip_pct < 1])

In [ ]:
agg_day_pivot_2023.columns

In [ ]:
agg_day_pivot_2023.columns